In [12]:
import argparse
import os
import pickle
import random
import shutil
import time

import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import sklearn.metrics as metrics
import torch
import torch.nn as nn
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from tensorboardX import SummaryWriter
from torch.autograd import Variable
from torch_geometric.utils import to_networkx

import configs
import gengraph
import models
import utils.featgen as featgen
import utils.graph_utils as graph_utils
import utils.io_utils as io_utils
import utils.math_utils as math_utils
import utils.parser_utils as parser_utils
import utils.train_utils as train_utils
from dataset import GraphDataset, create_dataset

In [13]:
G, labels, name = gengraph.gen_syn1(
    feature_generator=featgen.ConstFeatureGen(np.ones(32, dtype=float))
)
num_classes = max(labels) + 1

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32


<Figure size 2400x1800 with 0 Axes>

In [14]:
print(G[0])

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 12: {}, 15: {}, 17: {}, 28: {}, 29: {}, 30: {}, 31: {}, 35: {}, 39: {}, 44: {}, 47: {}, 53: {}, 63: {}, 66: {}, 71: {}, 76: {}, 77: {}, 78: {}, 80: {}, 92: {}, 95: {}, 96: {}, 99: {}, 110: {}, 115: {}, 116: {}, 126: {}, 147: {}, 155: {}, 163: {}, 165: {}, 166: {}, 174: {}, 185: {}, 196: {}, 200: {}, 203: {}, 208: {}, 225: {}, 231: {}, 232: {}, 236: {}, 250: {}, 263: {}, 269: {}, 272: {}, 276: {}, 300: {}}


In [20]:
# print(G[0])
# print((labels))

pyg_dataset = pyg_dataset = GraphDataset(
    torch.load("my_data/cycle_line_star_complete_1.pt", weights_only=False)
)

print(pyg_dataset[0])

for data in pyg_dataset:
    G = to_networkx(data, to_undirected=True)

    # Skip graphs without edges
    if G.number_of_edges() == 0:
        print(f"Graph {data} has no edges, skipping.")
        continue

Data(x=[9, 4], edge_index=[2, 58], y=[1], num_nodes=9, node_classes=[9])


In [19]:
graphs = []
all_labels = set()
for data in pyg_dataset:
    G = to_networkx(data, to_undirected=True)

    # Add graph label
    label = data.y.item()
    G.graph["label"] = label
    all_labels.add(label)

    # Add node features
    for i, node in enumerate(G.nodes()):
        G.nodes[node]["feat"] = data.x[i].numpy()

    graphs.append(G)

print(graphs[0])

for i, G in enumerate(graphs):
    print(f"Graph {i}: {G.number_of_edges()} edges")
    if G.number_of_edges() == 0:
        print(f"Adjacency matrix for graph {i}:\n", nx.adjacency_matrix(G).todense())

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}}
Graph 0: 29 edges
Graph 1: 30 edges
Graph 2: 31 edges
Graph 3: 30 edges
Graph 4: 32 edges
Graph 5: 34 edges
Graph 6: 31 edges
Graph 7: 34 edges
Graph 8: 37 edges
Graph 9: 29 edges
Graph 10: 30 edges
Graph 11: 31 edges
Graph 12: 30 edges
Graph 13: 32 edges
Graph 14: 34 edges
Graph 15: 31 edges
Graph 16: 34 edges
Graph 17: 37 edges
Graph 18: 29 edges
Graph 19: 30 edges
Graph 20: 31 edges
Graph 21: 30 edges
Graph 22: 32 edges
Graph 23: 34 edges
Graph 24: 31 edges
Graph 25: 34 edges
Graph 26: 37 edges
Graph 27: 29 edges
Graph 28: 30 edges
Graph 29: 31 edges
Graph 30: 30 edges
Graph 31: 32 edges
Graph 32: 34 edges
Graph 33: 31 edges
Graph 34: 34 edges
Graph 35: 37 edges
Graph 36: 29 edges
Graph 37: 30 edges
Graph 38: 31 edges
Graph 39: 30 edges
Graph 40: 32 edges
Graph 41: 34 edges
Graph 42: 31 edges
Graph 43: 34 edges
Graph 44: 37 edges
Graph 45: 29 edges
Graph 46: 30 edges
Graph 47: 31 edges
Graph 48: 30 edges
Graph 49: 32 edges
Graph 50: 